In [1]:
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt

In [2]:
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from os import listdir

2025-10-21 22:49:08.263482: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-21 22:49:09.702562: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-21 22:49:09.702708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-21 22:49:09.920449: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-21 22:49:10.490683: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-21 22:49:10.501657: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
damagepath = "/code/Project2/COE379L-Project2/damage"
nodamagepath = "/code/Project2/COE379L-Project2/no_damage"

X = []
y = []

# Load damaged images and convert to black and white
for img_name in os.listdir(damagepath):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(damagepath, img_name)
        img = load_img(img_path)
        img_gray = img.convert('L')
        img_array = img_to_array(img_gray)
        X.append(img_array)
        y.append(1)

# Load non-damaged images
for img_name in os.listdir(nodamagepath):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(nodamagepath, img_name)
        img = load_img(img_path)
        img_gray = img.convert('L')
        img_array = img_to_array(img_gray)
        X.append(img_array)
        y.append(0) 

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

print("Loaded images:", X.shape)
print("Labels:", y.shape)

Loaded images: (21322, 128, 128, 1)
Labels: (21322,)


In [4]:
print("Total images:", X.shape)
print("Total labels:", y.shape)

Total images: (21322, 128, 128, 1)
Total labels: (21322,)


In [5]:
print("Image shape (Height x Width x Channels):", X[0].shape)
print("All image shapes unique:", np.unique([img.shape for img in X], axis=0))

Image shape (Height x Width x Channels): (128, 128, 1)
All image shapes unique: [[128 128   1]]


In [6]:
print("Min pixel value:", X.min())
print("Max pixel value:", X.max())

Min pixel value: 0.0
Max pixel value: 255.0


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 43, stratify = y)

In [8]:
# Now I will scale the image to a 1D array 


[[[[140.]
   [141.]
   [141.]
   ...
   [140.]
   [142.]
   [143.]]

  [[142.]
   [142.]
   [142.]
   ...
   [140.]
   [142.]
   [143.]]

  [[141.]
   [141.]
   [141.]
   ...
   [141.]
   [141.]
   [142.]]

  ...

  [[ 96.]
   [ 93.]
   [ 93.]
   ...
   [145.]
   [145.]
   [145.]]

  [[ 94.]
   [ 90.]
   [ 90.]
   ...
   [147.]
   [147.]
   [146.]]

  [[104.]
   [ 94.]
   [ 85.]
   ...
   [150.]
   [149.]
   [148.]]]


 [[[ 90.]
   [ 91.]
   [ 93.]
   ...
   [ 92.]
   [ 93.]
   [ 93.]]

  [[ 91.]
   [ 91.]
   [ 93.]
   ...
   [ 91.]
   [ 91.]
   [ 91.]]

  [[ 91.]
   [ 92.]
   [ 92.]
   ...
   [ 90.]
   [ 90.]
   [ 90.]]

  ...

  [[141.]
   [129.]
   [144.]
   ...
   [ 73.]
   [ 77.]
   [ 67.]]

  [[130.]
   [119.]
   [134.]
   ...
   [ 67.]
   [ 75.]
   [ 70.]]

  [[135.]
   [127.]
   [137.]
   ...
   [ 59.]
   [ 71.]
   [ 69.]]]


 [[[131.]
   [130.]
   [129.]
   ...
   [ 59.]
   [ 59.]
   [ 59.]]

  [[127.]
   [127.]
   [129.]
   ...
   [ 66.]
   [ 64.]
   [ 62.]]

  [[122.]
   [12